In [1]:
import sympy as sp
import numpy as np
import scipy.optimize as opt
from copy import copy
import random as rd
from matplotlib import pyplot as plt
sp.init_printing(use_latex="mathjax")

# Aplicar ponto numa função vetorial e retornar o vetor
def point_in_func (func: list, symbols: list, point: list):
    func_in_point = []
    for i in range(0, len(func)):
        func_in_point.append(copy(func[i]))
        for j in range (0, len(symbols)):
            func_in_point[i] = func_in_point[i].subs(symbols[j], point[j])
            
    return func_in_point

# Retornar o produto escalar de dois vetores
def dot_product (v1: list, v2: list):
    result = 0.0
    
    if len(v1) == len (v2):
        for i in range (0, len(v1)):
            result += v1[i]*v2[i]
    
    return result

# Retornar a norma ao quadrado de um vetor
def squared_norm (Xn: list):
    return dot_product(Xn, Xn)

# Retornar o vetor v1-v2
def vec_sub (v1: list, v2: list):
    result = []
    if (len(v1) == len (v2)):
        for i in range (0, len(v1)):
            result.append(v1[i]-v2[i])
    
    return result

# Retornar novo vetor Xn
def get_new_xk (xk: list, step_size, direction: list):
    new_xk = []
    for i in range (0, len(xk)):
        new_xk.append( xk[i] + step_size*direction[i] )
    return new_xk

Digite quantas dimensões tem o domínio da função:

In [2]:
n = 2

In [3]:
# Criando as variáveis:
x = sp.symbols('x:'+ str(n))
display (x)

# Criando variável direção de busca:
d = sp.symbols('d:'+ str(n))
display (d)

(x₀, x₁)

(d₀, d₁)

Digite as funções fx (derivável) e gx (convexa) usando x[i] para cada coordenada i de um elemento x do domínio:

In [4]:
fx = [x[1]**2, x[0]**2, x[0]**2 + x[1]**2]
gx = [sp.Abs(x[1]), sp.Abs(x[0]), sp.Abs(x[0]) + sp.Abs(x[1])]

In [5]:
# Gerando Fx = fx + gx

m = len (fx)

Fx = []
for i in range (0, m):
    Fx.append(fx[i] + gx[i])
print ("Fx:")
display (Fx)

Fx:


⎡  2           2           2     2              ⎤
⎣x₁  + │x₁│, x₀  + │x₀│, x₀  + x₁  + │x₀│ + │x₁│⎦

In [6]:
# Calculando a função psix_d_i = grad(fx_i)*d + g_i(x+d) - g_i(x), para todo i

# Calculando a jacobiana de fx:
fx = sp.Matrix(fx)
print ("fx:")
display (fx)
Jfx = fx.jacobian(x)
print ("JFx (Jacobiana de fx):")
display (Jfx)

# Construindo g(x+d):
x_plus_d = []
for i in range (0,n):
    x_plus_d.append(d[i]+x[i])
gx_d = point_in_func (gx, x, x_plus_d)
print ("g(x+d):")
display(gx_d)

# Construindo pré psix(d):
pre_psix_d = []
for i in range (0,m):
    pre_psix_d.append(dot_product(Jfx.row(i),d) + gx_d[i] - gx[i])
print ("psix(d):")
display(pre_psix_d)

fx:


⎡     2   ⎤
⎢   x₁    ⎥
⎢         ⎥
⎢     2   ⎥
⎢   x₀    ⎥
⎢         ⎥
⎢  2     2⎥
⎣x₀  + x₁ ⎦

JFx (Jacobiana de fx):


⎡ 0    2⋅x₁⎤
⎢          ⎥
⎢2⋅x₀   0  ⎥
⎢          ⎥
⎣2⋅x₀  2⋅x₁⎦

g(x+d):


[│d₁ + x₁│, │d₀ + x₀│, │d₀ + x₀│ + │d₁ + x₁│]

psix(d):


[2⋅d₁⋅x₁ - │x₁│ + │d₁ + x₁│, 2⋅d₀⋅x₀ - │x₀│ + │d₀ + x₀│, 2⋅d₀⋅x₀ + 2⋅d₁⋅x₁ - │
x₀│ - │x₁│ + │d₀ + x₀│ + │d₁ + x₁│]

In [11]:
# A função objetivo phi(d) do subproblema para encontrar a direção dk:
def obj_func (d: list):
    # Retorna psix(d) aplicada em d 
    pre_psix_d_in_x = pre_psix_d_in_x_func(d)
    global l # Referenciamos l de fora da função
    return max(pre_psix_d_in_x) + (l/2)*squared_norm (d)

In [32]:
# Método Proximal-Gradiente Multiobjetivo (Rn -> Rm)

error = 10**(-6)
l = 0.1 # Constante positiva para minimizar o sobproblema e encontrar a direção do passo
p = 0.5 # Constante entre 0 e 1 para realizar a Regra de Armijo
E = 0.5 # Constante entre 0 e 1 pela qual multiplicamos o tamanho do passo para diminuí-lo

q_walks = 50
all_walks = []
for i_walk in range(0, q_walks):

    new_xk = []
    for j in range (0, n):
            new_xk.append((2*rd.random()-1.0)*100) # r.random() retorna um real entre 0 e 1
    
    print ("xk =", new_xk)       
    walk = [] # Lista com todos os pontos gerados no caminhamento
    while True:
        xk = new_xk # Se xk != new_xk, xk recebe new_xk e fazemos mais um passo

        # Obtendo Fx(xk)
        Fx_in_xk = point_in_func(Fx, x, xk) # Fx aplicada em xk
        walk.append(Fx_in_xk) # Salvando o caminhamento

        # Computando a direção de minimização:

        # Gerando a função phi(d) para otimizar como subproblema e encontrar a direção d
        pre_psix_d_in_x = point_in_func(pre_psix_d, x, xk) # Aplicando xk para criar a função objetivo
        pre_psix_d_in_x_func = sp.lambdify([d], pre_psix_d_in_x,'numpy') # Criando a função lambda

        display (pre_psix_d_in_x)

        # Criando um d0 inicial aleatório para a otimização quadrática de phi(d)
        d0 = []
        for j in range (0, n):
            d0.append((2*rd.random()-1.0)) # r.random() retorna um real entre 0 e 1

        # Realizamos a minimização de phi(d) e obtemos a direção de minimizaçaõ dk
        dk = opt.minimize(obj_func, d0, method='Powell').x

        print ("dk =",dk)

        # Computando o tamanho do passo:

        # Obtendo psi_xk(dk)
        psi_x_in_dk = max(pre_psix_d_in_x_func(dk)) # Pegamos o máximo de psi_xk(dk)    

        tk=1 # O tamanho do passo começa com 1
        while (True):
            # Preparando para verificar a Regra de Armijo
            new_xk = get_new_xk(xk, tk, dk) # new_xk = xk + tk*dk
            Fx_in_new_xk = point_in_func(Fx, x, new_xk) # Fx aplicada em new_xk

            print ("Fx(xk) =", Fx_in_xk, "Fx(xk+1) =", Fx_in_new_xk, "tk =", tk)

            tk_is_valid = True
            for i in range(0, m):
                # Se não respeitar a regra de armijo, para de verificar e indica que é inválido
                if Fx_in_new_xk[i] > Fx_in_xk[i] + tk*p*psi_x_in_dk: 
                    tk_is_valid = False
                    break

            # Se tk for válido para a Regra de Armijo, usamos ele
            if tk_is_valid:
                break
            else:
                tk = E*tk # Diminuímos tk se não for válido

        # Ao chegar aqui, o new_xk calculado respeita a Regra de Armijo

        # Paramos quando a diferença do x entre o passo atual e o anterior é muito pequena
        if (squared_norm(vec_sub(xk,new_xk)) <= error**2):
            break

        # Printando o new_xk encontrado e a norma quadrada da diferença com o xk 
        print ('new_xk =',new_xk,', ||xk-new_xk||² =',squared_norm(vec_sub(xk,new_xk)))

    print ("x =",new_xk)
    all_walks.append(walk)

xk = [-43.7975513635392, 72.20347114649701]


[144.406942292994⋅d₁ + │d₁ + 72.203471146497│ - 72.203471146497, -87.595102727
0784⋅d₀ + │d₀ - 43.7975513635392│ - 43.7975513635392, -87.5951027270784⋅d₀ + 1
44.406942292994⋅d₁ + │d₀ - 43.7975513635392│ + │d₁ + 72.203471146497│ - 116.00
1022510036]

dk = [ 0.34676388 -0.21128002]
Fx(xk) = [5285.54471674952, 1962.02305680539, 7247.56777355492] Fx(xk+1) = [5254.86777417572, 1931.42172019415, 7186.28949436987] tk = 1
new_xk = [-43.45078748093317, 71.99219112549794] , ||xk-new_xk||² = 0.16488443755337348


[143.984382250996⋅d₁ + │d₁ + 71.9921911254979│ - 71.9921911254979, -86.9015749
618663⋅d₀ + │d₀ - 43.4507874809332│ - 43.4507874809332, -86.9015749618663⋅d₀ +
 143.984382250996⋅d₁ + │d₀ - 43.4507874809332│ + │d₁ + 71.9921911254979│ - 115
.442978606431]

dk = [ 715.48370526 -429.44616305]
Fx(xk) = [5254.86777417572, 1931.42172019415, 7186.28949436987] Fx(xk+1) = [128130.796016662, 452300.275494015, 580431.071510677] tk = 1
Fx(xk) = [5254.86777417572, 1931.42172019415, 7186.28949436987] Fx(xk+1) = [20514.8379646612, 99093.1646971199, 119608.002661781] tk = 0.5
Fx(xk) = [5254.86777417572, 1931.42172019415, 7186.28949436987] Fx(xk+1) = [1286.36024338489, 18474.0341405502, 19760.3943839351] tk = 0.25
Fx(xk) = [5254.86777417572, 1931.42172019415, 7186.28949436987] Fx(xk+1) = [353.619550416234, 2160.57507273478, 2514.19462315101] tk = 0.125
Fx(xk) = [5254.86777417572, 1931.42172019415, 7186.28949436987] Fx(xk+1) = [2083.83738511422, 2.87209144441393, 2086.70947655864] tk = 0.0625
new_xk = [1.2669440977048296, 45.15180593486114] , ||xk-new_xk||² = 2720.081794720874


...

dk = [0.02772382 0.00018994]
Fx(xk) = [6011.21700037600, 0.0284924261807689, 6011.24549280218] Fx(xk+1) = [6011.18754764690, 5.72125163544690e-10, 6011.18754764747] tk = 1
new_xk = [5.72125163217363e-10, -77.03346082593563] , ||xk-new_xk||² = 0.0007686460916528009


[-154.066921651871⋅d₁ + │d₁ - 77.0334608259356│ - 77.0334608259356, 1.14425032
643473e-9⋅d₀ + │d₀ + 5.72125163217363e-10│ - 5.72125163217363e-10, 1.144250326
43473e-9⋅d₀ - 154.066921651871⋅d₁ + │d₀ + 5.72125163217363e-10│ + │d₁ - 77.033
4608259356│ - 77.0334608265078]

dk = [-5.72118917e-10  5.82922236e-04]
Fx(xk) = [6011.18754764690, 5.72125163544690e-10, 6011.18754764747] Fx(xk+1) = [6011.09715602998, 6.24616425516674e-15, 6011.09715602998] tk = 1
new_xk = [6.246164255166698e-15, -77.03287790369953] , ||xk-new_xk||² = 3.397983333468986e-07


[-154.065755807399⋅d₁ + │d₁ - 77.0328779036995│ - 77.0328779036995, 1.24923285
103334e-14⋅d₀ + │d₀ + 6.2461642551667e-15│ - 6.2461642551667e-15, 1.2492328510
3334e-14⋅d₀ - 154.065755807399⋅d₁ + │d₀ + 6.2461642551667e-15│ + │d₁ - 77.0328
779036995│ - 77.0328779036995]

dk = [-6.24616422e-15  6.33130683e-17]
Fx(xk) = [6011.09715602998, 6.24616425516674e-15, 6011.09715602998] Fx(xk+1) = [6011.09715602998, 3.56868138522202e-23, 6011.09715602998] tk = 1
x = [3.568681385222018e-23, -77.03287790369953]


In [33]:
# Para abrir a janela interativa do matplotlib, use:
%matplotlib qt 

# Plot 2D para funções com domínio R1
if m == 2:
    plt.figure()
    plt.xlabel("F1(x)")
    plt.ylabel("F2(x)")
    
    for i_walk in range (0, q_walks):
        walk_x_axis = []
        walk_y_axis = []

        # Plotando as caminhadas
        for j in range (0,len(all_walks[i_walk])):
            walk_x_axis.append(all_walks[i_walk][j][0])
            walk_y_axis.append(all_walks[i_walk][j][1])
        plt.plot (walk_x_axis, walk_y_axis, marker = 'o', linestyle = '-')
    
    # Plotando:
    plt.show()

# Plot 3D para funções com domínio R2
elif m == 3:
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_xlabel('f1')
    ax.set_ylabel('f2')
    ax.set_zlabel('f3')
    
    for i_walk in range (0, q_walks):
        walk_x_axis = []
        walk_y_axis = []
        walk_z_axis = []

        # Plotando as caminhadas
        for j in range (0,len(all_walks[i_walk])):
            walk_x_axis.append(all_walks[i_walk][j][0])
            walk_y_axis.append(all_walks[i_walk][j][1])
            walk_z_axis.append(all_walks[i_walk][j][2])
        ax.plot (walk_x_axis, walk_y_axis, marker = 'o', linestyle = '-')
    
    # Plotando:
    plt.show()